In [1]:
#import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
#spark_version = 'spark-3.3.1'
#os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
#!apt-get update
#!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
#!tar xf $SPARK_VERSION-bin-hadoop3.tgz
#!pip install -q findspark

# Set Environment Variables
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
#os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
#import findspark
#findspark.init(spark_home='/content/spark-3.3.1-bin-hadoop3')

Default spark install from starter did not work. py4j not found. installed default spark below.

In [2]:
# Import packages
!pip install pyspark -q
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
home_df.createOrReplaceTempView("home_sales")

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
average_price_four_bedroom_query = """
                                   SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS average_price
                                   FROM home_sales
                                   WHERE bedrooms = 4
                                   GROUP BY year
                                   ORDER BY year
                                   """
average_price_four_bedroom = spark.sql(average_price_four_bedroom_query)
average_price_four_bedroom.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
average_price_three_bedroom_three_bathroom_query = """
                                                   SELECT date_built, ROUND(AVG(price), 2) AS average_price
                                                   FROM home_sales
                                                   WHERE bedrooms = 3 AND bathrooms = 3
                                                   GROUP BY date_built
                                                   ORDER BY date_built
                                                   """
average_price_three_bedroom_three_bathroom = spark.sql(average_price_three_bedroom_three_bathroom_query)
average_price_three_bedroom_three_bathroom.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
average_price_three_bedroom_three_bathroom_two_floors_query = """
                                                              SELECT date_built, ROUND(AVG(price), 2) AS average_price
                                                              FROM home_sales
                                                              WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
                                                              GROUP BY date_built
                                                              ORDER BY date_built
                                                              """
average_price_three_bedroom_three_bathroom_two_floors = spark.sql(average_price_three_bedroom_three_bathroom_two_floors_query)
average_price_three_bedroom_three_bathroom_two_floors.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

view_rating_expensive_homes_query = """
                                    SELECT view
                                    FROM home_sales
                                    WHERE price >= 350000
                                    """
view_rating_expensive_homes = spark.sql(view_rating_expensive_homes_query)
view_rating_expensive_homes.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+
|view|
+----+
|  76|
|  23|
|   0|
|   4|
|   6|
|   7|
|  26|
|  34|
|  23|
|  48|
|  16|
|  10|
|  24|
|  91|
|  49|
|  37|
|  23|
|  25|
|  23|
|   1|
+----+
only showing top 20 rows

--- 0.2434859275817871 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

view_rating_expensive_homes_cashed = spark.sql(view_rating_expensive_homes_query)
view_rating_expensive_homes_cashed.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+
|view|
+----+
|  76|
|  23|
|   0|
|   4|
|   6|
|   7|
|  26|
|  34|
|  23|
|  48|
|  16|
|  10|
|  24|
|  91|
|  49|
|  37|
|  23|
|  25|
|  23|
|   1|
+----+
only showing top 20 rows

--- 2.056347131729126 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_df.write.partitionBy("date_built").parquet("home_sales_partitioned_parquet", mode='overwrite')

In [13]:
# 11. Read the parquet formatted data.
home_df_parquet = spark.read.parquet("home_sales_partitioned_parquet")

In [14]:
# 12. Create a temporary table for the parquet data.
home_df_parquet.createOrReplaceTempView("home_sales_partitioned")

In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

view_rating_expensive_homes_parquet = spark.sql(view_rating_expensive_homes_query)
view_rating_expensive_homes_parquet.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+
|view|
+----+
|  76|
|  23|
|   0|
|   4|
|   6|
|   7|
|  26|
|  34|
|  23|
|  48|
|  16|
|  10|
|  24|
|  91|
|  49|
|  37|
|  23|
|  25|
|  23|
|   1|
+----+
only showing top 20 rows

--- 0.1665644645690918 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
home_df.unpersist()

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [17]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False

In [18]:
!zip -r home_sales_partitioned_parquet.zip /content/home_sales_partitioned_parquet -q